In [ ]:
import cv2
import numpy as np
import glob

# Chessboard settings (adjust based on your board)
chessboard_size = (10, 7)  # 11x8 chessboard dimensions
square_size = 25  # in mm (for real-world scale, optional)

# Prepare object points (3D)
objp = np.zeros((np.prod(chessboard_size), 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2) * square_size

objpoints = []  # 3D world points
imgpoints = []  # 2D image points

# Load calibration images
images = glob.glob("Calibration_Images/*.jpg")  # Only JPG  # Match all files # Path to saved images

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and show corners
        cv2.drawChessboardCorners(img, chessboard_size, corners, ret)
        cv2.imshow('Corners', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

# Camera Calibration
ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Save calibration results
np.savez("camera_calibration.npz", camera_matrix=camera_matrix, dist_coeffs=dist_coeffs)

print("Camera Matrix:\n", camera_matrix)
print("Distortion Coefficients:\n", dist_coeffs)

In [ ]:
total_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], camera_matrix, dist_coeffs)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
    total_error += error
print("Mean Reprojection Error:", total_error / len(objpoints))

In [ ]:
# Load camera calibration parameters
calibration_data = np.load("camera_calibration.npz")
camera_matrix = calibration_data["camera_matrix"]
dist_coeffs = calibration_data["dist_coeffs"]

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import shutil

# Define folder name
frame_save_path = "captured_frames_IONI"

# Delete the folder if it exists and create a new one
if os.path.exists(frame_save_path):
    shutil.rmtree(frame_save_path)

os.makedirs(frame_save_path)

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Eye landmark indices (MediaPipe 468-face model)
RIGHT_EYE = [362, 385, 387, 263, 373, 380]  # Left eye landmarks
LEFT_EYE = [33, 160, 158, 133, 153, 144]  # Right eye landmarks

# Eye aspect ratio (EAR) threshold for blink detection
EAR_THRESHOLD = 0.18  # Lowered threshold for better sensitivity
BLINK_CONSECUTIVE_FRAMES = 2  # Fewer frames to confirm a blink

# Variables for blink detection
left_eye_closed = False
right_eye_closed = False
left_eye_counter = 0
right_eye_counter = 0

# Data storage for graphs
gaze_data = []  # Stores gaze direction (x, y)
frame_numbers = []  # Stores frame numbers
all_dilation_data = []  # Stores pupil sizes over time
all_frame_numbers = []  # Stores corresponding frame numbers
left_pupil_coords = []  # Stores left pupil coordinates
right_pupil_coords = []  # Stores right pupil coordinates
left_blink_frames = []  # Stores frames where left eye blinks
right_blink_frames = []  # Stores frames where right eye blinks
both_blink_frames = []  # Stores frames where both eyes blink

# Gaze direction classification (8 classes)
DIRECTIONS_8 = ["Right", "Top-Right", "Top", "Top-Left", "Left", "Bottom-Left", "Bottom", "Bottom-Right"]

def normalized_to_pixel_coords(landmark, image_width, image_height):
    """Convert normalized landmark coordinates to pixel values"""
    return int(landmark.x * image_width), int(landmark.y * image_height)

def extract_eye_region(frame, eye_landmarks):
    """Crop the eye region from the frame"""
    mask = np.zeros_like(frame[:, :, 0])
    points = np.array([normalized_to_pixel_coords(l, frame.shape[1], frame.shape[0]) for l in eye_landmarks])

    cv2.fillPoly(mask, [points], 255)
    eye = cv2.bitwise_and(frame, frame, mask=mask)

    x, y, w, h = cv2.boundingRect(points)
    eye_cropped = eye[y:y+h, x:x+w]

    return eye_cropped, x, y, w, h

def detect_pupil(eye):
    """Detect the pupil using adaptive thresholding, contour detection, and Hough Circles"""
    if eye is None or eye.size == 0:
        return None, None, None
    # Convert to grayscale and enhance contrast
    gray = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)

    # Adaptive thresholding
    thresh = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
    )

    # Morphological operations to remove noise
    kernel = np.ones((3, 3), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Filter contours by size
        contours = [c for c in contours if 100 < cv2.contourArea(c) < 5000]

        if contours:
            # Find the largest contour
            largest_contour = max(contours, key=cv2.contourArea)

            # Fit an ellipse to the largest contour
            if len(largest_contour) >= 5:
                ellipse = cv2.fitEllipse(largest_contour)
                (px, py), (MA, ma), angle = ellipse
                radius = (MA + ma) / 4  # Approximate radius from major/minor axis
                return int(px), int(py), int(radius)

    # Fallback: Use Hough Circles if contour detection fails
    circles = cv2.HoughCircles(
        blurred,
        cv2.HOUGH_GRADIENT,
        dp=1,
        minDist=50,
        param1=50,
        param2=30,
        minRadius=10,
        maxRadius=50
    )

    if circles is not None:
        circles = np.uint16(np.around(circles))
        for circle in circles[0, :]:
            px, py, radius = circle[0], circle[1], circle[2]
            return px, py, radius

    return None, None, None

def eye_aspect_ratio(eye_landmarks):
    """Calculate the eye aspect ratio (EAR) to detect blinks"""
    # Extract landmark coordinates
    points = np.array([(lm.x, lm.y) for lm in eye_landmarks])

    # Compute the Euclidean distances between the vertical eye landmarks
    A = np.linalg.norm(points[1] - points[5])
    B = np.linalg.norm(points[2] - points[4])

    # Compute the Euclidean distance between the horizontal eye landmarks
    C = np.linalg.norm(points[0] - points[3])

    # Calculate the EAR
    ear = (A + B) / (2.0 * C)
    return ear

def normalize_gaze_vector(gaze_vector):
    """Normalize the gaze vector to a unit vector."""
    magnitude = np.linalg.norm(gaze_vector)
    if magnitude == 0:
        return gaze_vector  # Avoid division by zero
    return gaze_vector / magnitude

def classify_gaze_direction(gaze_vector):
    """Classify gaze direction into 8 classes based on the angle of the gaze vector."""
    angle = np.arctan2(gaze_vector[1], gaze_vector[0])  # Angle in radians
    angle_deg = np.degrees(angle) % 360  # Convert to degrees and normalize to [0, 360)

    # Define angle ranges for 8 directions
    if 337.5 <= angle_deg or angle_deg < 22.5:
        return "Right"
    elif 22.5 <= angle_deg < 67.5:
        return "Top-Right"
    elif 67.5 <= angle_deg < 112.5:
        return "Top"
    elif 112.5 <= angle_deg < 157.5:
        return "Top-Left"
    elif 157.5 <= angle_deg < 202.5:
        return "Left"
    elif 202.5 <= angle_deg < 247.5:
        return "Bottom-Left"
    elif 247.5 <= angle_deg < 292.5:
        return "Bottom"
    elif 292.5 <= angle_deg < 337.5:
        return "Bottom-Right"
    

def get_red_cross_position(frame_count, frame_width, frame_height, speed=10, padding=50):
    """
    Calculate the position of the red cross based on the frame count.
    The cross moves in a rectangular path along the border of the screen with equal padding.
    Ensures the cross stays within the padded area.
    """
    # Calculate the dimensions of the inner rectangle
    inner_width = frame_width - 2 * padding
    inner_height = frame_height - 2 * padding

    # Total perimeter of the inner rectangle
    perimeter = 2 * (inner_width + inner_height)

    # Distance traveled by the cross
    distance = (frame_count * speed) % perimeter

    # Calculate the position of the red cross
    if distance < inner_width:
        # Moving along the top edge (with padding)
        x = padding + distance
        y = padding
    elif distance < inner_width + inner_height:
        # Moving along the right edge (with padding)
        x = frame_width - padding - 1
        y = padding + (distance - inner_width)
    elif distance < 2 * inner_width + inner_height:
        # Moving along the bottom edge (with padding)
        x = padding + (inner_width - (distance - (inner_width + inner_height)))
        y = frame_height - padding - 1
    else:
        # Moving along the left edge (with padding)
        x = padding
        y = padding + (inner_height - (distance - (2 * inner_width + inner_height)))

    # Clamp the coordinates to ensure they stay within the padded area
    x = max(padding, min(x, frame_width - padding - 1))
    y = max(padding, min(y, frame_height - padding - 1))

    return int(x), int(y)

# Start Video Capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

# Get actual frame width & height
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"Using Webcam Resolution: {frame_width}x{frame_height}")

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.undistort(frame, camera_matrix, dist_coeffs)
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    # Get the position of the red cross
    red_cross_x, red_cross_y = get_red_cross_position(frame_count, frame_width, frame_height, speed=10, padding=40)

    # Draw the red cross on the frame
    cross_size = 20  # Size of the cross
    cv2.line(frame, (red_cross_x - cross_size, red_cross_y), (red_cross_x + cross_size, red_cross_y), (0, 0, 255), 2)
    cv2.line(frame, (red_cross_x, red_cross_y - cross_size), (red_cross_x, red_cross_y + cross_size), (0, 0, 255), 2)

    text_x, text_y = 50, 50  # Shift text to top

    cv2.putText(frame, "Gaze & Pupil Tracking", (text_x, text_y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    if results.multi_face_landmarks:
        face_landmarks = results.multi_face_landmarks[0]

        left_eye = [face_landmarks.landmark[i] for i in LEFT_EYE]
        right_eye = [face_landmarks.landmark[i] for i in RIGHT_EYE]

        

        

        

        

        

        

        

        

            

            

            

    

    
    
    
    















    

    
    

    

    

    

        

        

       

        

        

        

            

            

    

    
    
    
    














